# Raise base class events in derived classes

Let's create a smart kitchen that listens to multiple appliances in the room and logs what they're being used for.

In [ ]:
public class ProcessEventArgs : EventArgs {
    public string Message { get; init; }
    public DateTime TimeStamp { get; } = DateTime.Now;

    public TimeSpan GetTimeElapsed(ProcessEventArgs previousProcess) {
        return TimeStamp - previousProcess.TimeStamp;
    }
}

In [ ]:
/// Publisher base class
public abstract class Appliance {
    public List<ProcessEventArgs> ProcessesHistory { get; } = new();

    public event EventHandler<ProcessEventArgs> ProcessStartedEvent;

    /// Derived classes can override and call this method
    protected virtual void OnProcessStarted(ProcessEventArgs e) {
        ProcessesHistory.Add(e);
        // safely raise the event by using ?.Invoke
        ProcessStartedEvent?.Invoke(this, e);
    }
}

In [ ]:
public class Stove : Appliance {
    class Top {
        public int Index { get; init; }
        public string Something { get; set; }
        public bool IsBusy => !string.IsNullOrEmpty(Something);
    }

    private Top[] Tops { get; } = Enumerable.Range(1, 6)
        .Select(i => new Top {
            Index = i
        })
        .ToArray();
    
    public void Heat(string something, int onTop) {
        if (Tops[onTop].IsBusy) {
            Console.WriteLine($"Stove top {onTop} is busy right now.");
            return;
        }
        Tops[onTop].Something = something;

        var process = new ProcessEventArgs {
            Message = $"Start heating {something} at stove top {onTop}"
        };
        OnProcessStarted(process);
    }

    protected override void OnProcessStarted(ProcessEventArgs e) {
        base.OnProcessStarted(e);
    }
}

In [ ]:
public class SmartKitchen {
    public List<Appliance> Appliances { get; } = new();

    public void AddApliance(Appliance appliance) {
        Appliances.Add(appliance);
        appliance.ProcessStartedEvent += HandleProcessStarted;
    }

    private void HandleProcessStarted(object sender, ProcessEventArgs e) {
        Console.WriteLine($"{sender.GetType()}:\n\t{e.Message}\n\t{e.TimeStamp}");
    }
}

In [ ]:
var stove = new Stove();
var smartKitchen = new SmartKitchen();
smartKitchen.AddApliance(stove);

stove.Heat("water", 0);

Submission#49+Stove:
	Start heating water at stove top 0
	1/22/2022 2:57:37 PM


In [ ]:
stove.Heat("soup", 0);

Stove top 0 is busy right now.


In [ ]:
stove.Heat("soup", 1);

Submission#49+Stove:
	Start heating soup at stove top 1
	1/22/2022 2:57:39 PM
